# Machine Learning Project
## Articles Analysis with Machine Learning

### Revisando a proposta

#### Background:

Todos os dias pesquisadores, estudantes de graduação, mestrado, doutorado, curiosos estão descobrindo e escrevendo sobre nossa visão do mundo, artigos sobre astronomia, psicologia, informática, engenharia mecânica, entre outros, questionam e encontram informações sobre pequenas, grandes, estranhas, importantes ou inúteis descobertas. Se separarmos uma certa área específica, será que uma pessoa é capaz de ler, entender, conversar com os autores daquele assunto? Num mundo tão grande, com uma produção tão vasta, é difícil.

Em 2011, por exemplo, os pesquisadores brasileiros publicaram 49.664 artigos 2, olhando o mapa acima, talvez no mesmo ano, os Estados Unidos tenham publicado cerca de 20 vezes mais. Mas a disputa aqui não é sobre quem publica mais, mas sim, sobre seria possível consumir uma quantidade grande de produção científica.

O trabalho acadêmico tem por critério de qualidade, geralmente, a originalidade do trabalho, os pesquisadores citam em seus projetos descobertas originais, ou aplicação de métodos em novos tipos de dados, entre outras maneiras, porém de alguma forma estes pesquisadores estão ligados, por seus assuntos ou por métodos utilizados, o mundo científico também adotou a interdisciplinaridade, por vezes alguns artigos de medicina irão usar técnicas de ciências da computação ou matemática para comprovar uma descoberta, a arte pode usar a física, a física pode usar a psicologia.

Então se alguém escreve, ou se vai escrever sobre um assunto, quem são as outras pessoas da área dela, ou não, que estão produzindo sobre assuntos que dividem características com as essas publicações, se uma universidade ou revista quiser saber quais autores tem mais similaridade nos métodos usados, ou nos dados pesquisados, como unir essa informação sem ter que ler toda a volumosa produção científica que o mundo produz?

#### Problema e Justificativa:

Analisar em que revistas este artigo tem mais probabilidade de sucesso de aceitação, sabendo quais grupos as revistas mais publicam. Este algoritmo poderia ser o mesmo utilizado para agrupar os dados, no entanto, para explorar mais ferramentas de aprendizado de máquina, será utilizado um algoritmo para classificação, utilizando como variável alvo o grupo encontrado na etapa de agrupamento. 

#### Solução Proposta:

A solução e objetivos propostos são:
Utilizar processamento de linguagem natural para ler resumos de artigos científicos e encontrar palavras importantes para classificar seus autores;
Clusterizar os autores e revistas onde os artigos foram publicados para encontrar grupos naturalmente formados;
Prever a possibilidade de um autor fazer parte de um grupo de autores ou de publicar em uma revista;

(O autor propôs esta solução para utilizar aprendizagem supervisionada e não supervisionada no exercício final).

#### Solução Proposta pelo Orientador:

Criar um algortimo de aprendizagem supervisionada e já utilizar as revistas como variável alvo, sem a necessidade de realizar clusterização.

### Iniciando o projeto

Alguns pacotes foram criados para auxiliar a execução do projeto, os mesmos podem ser acessados no diretório do github, todas as classes e scripts criados possuem docstring para detalhar a utilização dos mesmos. O objetivo era deixar no notebook apenas as partes mais importantes.

### Conjunto de dados e inputs

Os artigos foram coletados, manualmente, no site (https://www.ncbi.nlm.nih.gov/pubmed/). No entanto, em vez de utilizar os artigos no formato .nbib, como ideaizado no projeto, será utilizado arquivos .xml.

Eles estão no idioma inglês, para evitar erros de viés durante a elaboração dos algoritmos, todos os artigos deste website são sobre medicina, de modo que possamos reduzir o grupo de observação e não coletar dados extremamente distantes que não sejam efetivos no momento da construção do algoritmo.

In [1]:
import numpy as np
import pandas as pd

import os
import glob
import xml.etree.ElementTree

# Important nltk packages you might download to execute the codes
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('stopwords')

%matplotlib inline
import matplotlib.pyplot as plt

from custom_entities.article import ArticleData
from custom_helpers.pandas_helper import insert_article_to_pandas

# Path of folder with all articles
# path = 'C:/Users/Paulo/Documents/PyCharmProjects/papers_abstract_analysis/papers'
path = 'C:/Users/Paulo Henrique/PycharmProjects/papers_abstract_analysis/papers'

df = pd.DataFrame()

for filename in glob.glob(os.path.join(path, '*.xml')):
    root = xml.etree.ElementTree.parse(filename).getroot()
    article = ArticleData(root)
    df = insert_article_to_pandas(df, article)
    
df = df.fillna(0)

n_articles = len(df.index)
n_features = len(df.columns) - 1
# Imprime os resultados
print "Número total de artigos: {}".format(n_articles)
print "Número de atributos: {}\n".format(n_features)

for index, row in (df.groupby(['.JOURNAL']).count()).iterrows():
    print index, row[0]

Número total de artigos: 124
Número de atributos: 491

Basic & clinical pharmacology & toxicology 17
Child psychiatry and human development 15
Clinical psychology & psychotherapy 20
Drug and alcohol dependence 17
Journal of affective disorders 20
Neuropharmacology 17
Psychiatry investigation 18


## Classificador com revista como variável alvo
### Separação da variável alvo

Inicialmente será criado um classificador usando as revistas como variváveis alvo.

In [10]:
# Extrair coluna de atributos
feature_cols = list(df.columns[1:])

# Extrair coluna alvo
target_col = df.columns[0] 

# Mostre a lista de colunas
print "Colunas de atributos (Primeiras 40):\n{}".format(feature_cols[0:40])
print "\nColuna-alvo: {}".format(target_col)

# Separação do data_set de atributos e alvo
X_all = df[feature_cols]
y_all = df[target_col]

Colunas de atributos (Primeiras 40):
['abstin', 'access', 'accommod', 'acetaminophen', 'across', 'action', 'activ', 'addict', 'adolesc', 'adrenerg', 'adult', 'adulthood', 'affair', 'affect', 'agent', 'aggress', 'alcohol', 'alexithymia', 'allianc', 'almost', 'although', 'aluminium', 'amazon', 'ambival', 'among', 'analges', 'analysi', 'androstan', 'anesthet', 'anti-epilept', 'antibiot', 'antidepress', 'anxieti', 'anxiou', 'apach', 'aptam', 'arson', 'assert', 'assess', 'associ']

Coluna-alvo: .JOURNAL


### Divisão dos Dados de Treinamento e Teste

Embaralhar aleatoriamente os dados (X_all, y_all) em subconjuntos de treinamento e teste.
    > Utilizar 75% de dados em treinamento 25% em teste.
    > Armazenar os resultados em X_train, X_test, y_train e y_test.

In [3]:
# Importar divisor de amostra
from sklearn.cross_validation import train_test_split

# 25% para teste
num_test = 0.25
custom_random_state = 32

# Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=custom_random_state)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 93 amostras.
O conjunto de teste tem 31 amostras.


c:\python27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Utilidades para ajustar treinamento

Implemtanção de métodos para auxiliar o treinamento e visão de resultados

In [4]:
#implemtação de codigo para imprimir o markdown sozinho :)
from time import time
from sklearn.metrics import accuracy_score

global col_1
global col_2
global col_3
global col_4
global col_5
col_1 = 'Tamanho do Conj de Treinamento'
col_2 = 'Tempo de Treinamento'
col_3 = 'Tempo de Estimativa (teste)'
col_4 = 'Pontuacao Accuracy (treinamento)'
col_5 = 'Pontuacao Accuracy (teste)'

global df_model
df_model = pd.DataFrame(columns=[col_1, col_2, col_3, col_4, col_5])
df_backup = df_model.copy()

def pandas_df_to_markdown_table(df):
    '''
    Função para tabular com markdown.
    Creditos para o autor na publicação abaixo.
    Acessado em https://stackoverflow.com/questions/33181846/programmatically-convert-pandas-dataframe-to-markdown-table
    '''
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_csv(sep="|", index=False)))

In [5]:
def train_classifier(clf, X_train, y_train, index=None):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)
    if index:
        df_model.loc[index,col_2] = "{:.4f} seg".format(end - start)

    
def predict_labels(clf, features, target, index=None):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    if index:
        df_model.loc[index, col_3] = "{:.4f} seg".format(end - start)
    
    return accuracy_score(target.values, y_pred)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    index = len(X_train)
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, index)
    df_model.loc[index, col_1] = index
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train, index)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    time_train = predict_labels(clf, X_train, y_train, index)
    time_test = predict_labels(clf, X_test, y_test, index)
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(time_train)
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(time_test)
    df_model.loc[index, col_4] = "{:.4f}.".format(time_train)
    df_model.loc[index, col_5] = "{:.4f}.".format(time_test)

### Métricas de Desempenho do Modelo

Implemtanção de métodos para auxiliar o treinamento e visão de resultados.

In [6]:
# Importar três modelos de aprendizagem supervisionada do sklearn
# from sklearn import model_A
from sklearn.linear_model import LogisticRegression
# from sklearn import model_B
from sklearn.tree import DecisionTreeClassifier
# from skearln import model_C
from sklearn.svm import SVC

# TODO: Inicialize os três modelos
clf_A = LogisticRegression(random_state=custom_random_state)
clf_B = DecisionTreeClassifier(random_state=custom_random_state)
clf_C = SVC(random_state=custom_random_state)

# Cria array de classificadores e dataframe de resultados
clfs = [clf_A, clf_B, clf_C]
df_A = None
df_B = None
df_C = None
dfs = [df_A, df_B, df_C]

# Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
# train_predict(clf, X_train, y_train, X_test, y_test)
for i, x in enumerate(clfs):
    train_predict(clfs[i], X_train, y_train, X_test, y_test)
    print ''
    dfs[i] = df_model.copy()
    df_model = df_backup.copy()

Treinando um LogisticRegression com 93 pontos de treinamento. . .
O modelo foi treinado em 0.0030 segundos
As previsões foram feitas em 0.0000 segundos.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 0.9785.
Pontuação F1 para o conjunto de teste: 0.4194.

Treinando um DecisionTreeClassifier com 93 pontos de treinamento. . .
O modelo foi treinado em 0.0040 segundos
As previsões foram feitas em 0.0000 segundos.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 0.9785.
Pontuação F1 para o conjunto de teste: 0.4194.

Treinando um SVC com 93 pontos de treinamento. . .
O modelo foi treinado em 0.0090 segundos
As previsões foram feitas em 0.0040 segundos.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.1828.
Pontuação F1 para o conjunto de teste: 0.0968.



In [7]:
#Exibindo tabelas automaticamente
for i, x in enumerate(clfs):
    print '\033[1m' + '\033[4m' + "\nClassificador " + str(i+1) + ' - ' + clfs[i].__class__.__name__ + '\033[0m'
    pandas_df_to_markdown_table(dfs[i])


Classificador 1 - LogisticRegression


Tamanho do Conj de Treinamento|Tempo de Treinamento|Tempo de Estimativa (teste)|Pontuacao Accuracy (treinamento)|Pontuacao Accuracy (teste)
---|---|---|---|---
93|0.0030 seg|0.0000 seg|0.9785.|0.4194.



Classificador 2 - DecisionTreeClassifier


Tamanho do Conj de Treinamento|Tempo de Treinamento|Tempo de Estimativa (teste)|Pontuacao Accuracy (treinamento)|Pontuacao Accuracy (teste)
---|---|---|---|---
93|0.0040 seg|0.0000 seg|0.9785.|0.4194.



Classificador 3 - SVC


Tamanho do Conj de Treinamento|Tempo de Treinamento|Tempo de Estimativa (teste)|Pontuacao Accuracy (treinamento)|Pontuacao Accuracy (teste)
---|---|---|---|---
93|0.0090 seg|0.0010 seg|0.1828.|0.0968.


### Calibrando o Melhor Modelo

...

In [8]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer

# Novo embaralhamento para busca de parametros
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_all, y_all, test_size=num_test, random_state=custom_random_state)

# TODO: Crie a lista de parâmetros que você gostaria de calibrar
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
class_weight = ['balanced', None]
parameters = {'C': C,
              'class_weight': class_weight}

# TODO: Inicialize o classificador
clf = LogisticRegression(random_state=custom_random_state)

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(accuracy_score)

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, cv=5, verbose=0)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj.fit(X_train_cv, y_train_cv)

# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetros
tuned_f1_train = "{:.4f}".format(predict_labels(clf, X_train, y_train))
tuned_f1_test = "{:.4f}".format(predict_labels(clf, X_test, y_test))
print "\nO modelo calibrado tem Accuracy de " + tuned_f1_train + " no conjunto de treinamento."
print "O modelo calibrado tem Accuracy de " + tuned_f1_test + " no conjunto de teste."

print "\nMelhores parametros:"
print "C = " + str(grid_obj.best_params_['C'])
print "class_weight = " + str(grid_obj.best_params_['class_weight'])

df_tuned = dfs[0].copy()
df_tuned.loc[:, col_4] = tuned_f1_train
df_tuned.loc[:, col_5] = tuned_f1_test

c:\python27\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


As previsões foram feitas em 0.0010 segundos.
As previsões foram feitas em 0.0000 segundos.

O modelo calibrado tem Accuracy de 0.9462 no conjunto de treinamento.
O modelo calibrado tem Accuracy de 0.5161 no conjunto de teste.

Melhores parametros:
C = 0.1
class_weight = balanced


In [9]:
print '\033[1m' + '\033[4m' + "\nAntes: " + clfs[0].__class__.__name__ + '\033[0m'
pandas_df_to_markdown_table(dfs[0].loc[:,[col_1, col_4, col_5]])

print '\033[1m' + '\033[4m' + "\nDepois: " + clfs[0].__class__.__name__ + '\033[0m'
pandas_df_to_markdown_table(df_tuned.loc[:,[col_1, col_4, col_5]])


Antes: LogisticRegression


Tamanho do Conj de Treinamento|Pontuacao Accuracy (treinamento)|Pontuacao Accuracy (teste)
---|---|---
93|0.9785.|0.4194.



Depois: LogisticRegression


Tamanho do Conj de Treinamento|Pontuacao Accuracy (treinamento)|Pontuacao Accuracy (teste)
---|---|---
93|0.9462|0.5161


### END
